In [ ]:
# Instalar librerías, en caso de que no las tengan instaladas.
!pip install --upgrade pip
!pip install -r requirements.txt

In [ ]:
# Comandos import
import fasttext
import numpy
import gensim
import logging
import gensim.downloader as api
from scipy import spatial
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.test.utils import datapath
from gensim import utils
import os
from os import listdir
#nltk.download('stopwords')
from nltk.corpus import stopwords

# Algunas nociones básicas

**Hipótesis distribucional**: Las palabras que aparecen en los mismos contextos tienen significados similares.

**Semántica de vectores**: Semántica que implementa la hipótesis distribucional mediante un modelo de aprendizaje automático que construye representaciones del significado de las palabras de manera tal que cada palabra es un punto en un espacio multidimensional derivado de la distribución de las palabras vecinas en corpus. Cada punto se define mediante un vector que se denomina embedding.


El primer modelo de semántica de vectores surgió en el área de análisis de sentimiento. Cada paalbra se representaba como un vector en un espacio de tres dimensiones: valencia, *arousal*, y dominancia.

Los dos modelos más conocidos de semántica de vectores son tf-idf y word2vec.

# tf-idf

**matriz de coocurrencia**: modelo vectorial para representar cuan frecuentemente coocurren las palabras. Existen distintos tipos: matriz de términos-documentos, en que las filas son palabras y las columnas son documentos; matriz de término-término, en que cada celda indica la cantidad de veces que coocurren la palabra de la columna con la de la fila.

**tf**: frecuencia de un determinado término

**df**: cantidad de documentos en que aparece un término.

**idf**: número total de documentos dividido la cantidad de documentos en que aparece un término en cuestión.

In [ ]:
corpus = ['vegetables are good for health', 'fruits are good too',
'vegetables are good for health, fruits are good too and they are healthy']

def vectorizacion(corpus, language):
    vectorizer = TfidfVectorizer(stop_words=stopwords.words(language))
    tfidf_matrix = vectorizer.fit_transform(corpus).toarray()
    print('la cantidad de documentos y palabras a considerar son '+str(tfidf_matrix.shape)) # Cantidad de documentos y cantidad de palabras
    print('los términos a considerar son '+str(vectorizer.get_feature_names())) # Términos a considerar
    doc_number = 0
    for file in corpus:
        print(tfidf_matrix[doc_number])
        doc_number = doc_number + 1
    return doc_number, vectorizer
    
    
vectorizacion(corpus,'english')

In [ ]:
def comparison(corpus, query, language):
    doc_number, vectorizer = vectorizacion(corpus,language)
    print(doc_number)
#    print(vectorizer)
    tfidf_matrix = vectorizer.fit_transform(corpus).toarray()
#        vectorizer = TfidfVectorizer(stop_words=language)
    new_vector = vectorizer.transform(query).toarray().reshape(-1)
    reference_number = 0
    while reference_number <= int(doc_number)-1:
        print('tfidf'+str(reference_number)+', similarity score: ',spatial.distance.cosine(new_vector,tfidf_matrix[reference_number]))
        reference_number = reference_number + 1
        


In [ ]:
query = ['fruits and vegetables are good for health']
comparison(corpus,query,'english')


In [ ]:
# Esta función toma un directorio y devuelve una lista que contiene 
# todo archivo con una extensión particular en ese directorio
def upload_files(path, extension):
    filenames = listdir(path)
    fullnames = [ os.path.join(path, filenames) for filenames in filenames if os.path.join(path, filenames).endswith( extension ) ]
    return fullnames

# Esta función toma todo txt en un directorio que contiene al corpus  
# and returns the list of sentences from that corpus

def consider_corpus_files(path):
    sent_list = []
    corpus = []
    for file in upload_files(path, "txt"):
        fi = open(file, "r")
        corpus.append(fi.read())
    return corpus

consider_corpus_files('corpus/')

In [ ]:
vectorizacion(consider_corpus_files('corpus/'),'spanish')

In [ ]:
query = ['facundo y rosas']
comparison(consider_corpus_files('corpus/'),query,'spanish')

# Word2vec

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
wv = api.load('word2vec-google-news-300')

In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

In [ ]:
vec_car = wv['car']
print(vec_car)

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

In [ ]:
vec_king = wv['king']
print(vec_king)


In [ ]:
vec_male = wv['male']
print(vec_male)

In [ ]:
vec_woman = wv['woman']
print(vec_woman)

In [ ]:
vec_queen = wv['queen']
print(vec_queen)


In [ ]:
analogy = vec_king - vec_male + vec_woman
print(analogy)

Calcular similitud mediante cálculo del coseno

In [ ]:
# coseno entre king y queen y king y male usando gensim
pairs = [
    ('king', 'queen'),
    ('king', 'male'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

In [ ]:
# coseno entre king y queen usando scipy
result = 1 - spatial.distance.cosine(vec_king, vec_queen)
print(result)

In [ ]:
# coseno entre queen y queen-male+woman usando scipy
result = 1 - spatial.distance.cosine(vec_queen, analogy)
print(result)

In [ ]:
# coseno entre king y queen-male+woman usando scipy
result = 1 - spatial.distance.cosine(vec_king, analogy)
print(result)

Se puede calcular las 5 palabras más cercanas.

In [ ]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

El modelo no es capaz de inferir un vector para una palabra desconocida

In [ ]:
try:
    word = 'cameroon'
    vec_cameroon = wv[word]
except KeyError:
    print("The word "+word+" does not appear in this model")

Entrenar un modelo propio

In [ ]:
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = open('corpus/facundo.txt')
        for line in corpus_path:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

In [ ]:
vec_Facundo = model.wv['facundo']
print(vec_Facundo)

In [ ]:
print(model.wv.most_similar(positive=['rosas'], topn=20))

# Fasttext

In [ ]:
import fasttext
# Baja el modelo y lo divide
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
!head cooking.stackexchange.txt
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid


In [ ]:
model = fasttext.train_supervised(input="cooking.train")


In [ ]:
model.predict("Which baking dish is best to bake a banana bread ?")

In [ ]:
model.predict("Which fruit is sweet ?")